In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab10.ipynb")

<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2021<br>
            Dr. Eric Van Dusen<br>
        Shashank Dalmia<br></p></td></tr>
</table>

## Lab 10: Econometrics Part 1

In [2]:
from datascience import  *
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D 
import numpy as np
import pandas as pd
import statsmodels.api as sm
plt.style.use('seaborn-muted')

import warnings
warnings.filterwarnings("ignore")

## Part 0: Introduction

Stock valuations have been a great predictor of future returns. In this lab, we will take a look at how good valuations actually are and the limitations of this analysis. Also, we will learn regression in the process.

This [article](https://www.wsj.com/articles/the-economy-is-recovering-how-to-invest-when-everything-is-expensive-11618063500?mod=business_minor_pos10) in the Wall Street Journal is a great example of what we are going to be doing. (Also very timely as it was published just a few days before this lab was released.)

Note: UC Berkeley students have free access to the WSJ. But if you can't access the article, here is a [PDF](https://drive.google.com/file/d/1tbo6TRbP-KKSoVRVCqdHXOLQSeaY4J8n/view?usp=sharing).

![WSJ headline](headline.png)

The headline alludes to an idea that at high and expensive valuations, future returns in stock market will be lower. Lets try to test out the hypothesis.

First, lets load the data and take a look at it. This comes from a combination of [Yahoo Finance](https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC), [FRED](https://fred.stlouisfed.org/graph/?g=qis), and [Nobel laureate Prof Robert Schiller](http://www.econ.yale.edu/~shiller/data.htm).

**Disclamer:** This lab does not provide any investment advice.

In [3]:
data = Table().read_table("data.csv")
data

The data contains quartely S&P 500 closing price. The [S&P 500](https://en.wikipedia.org/wiki/S%26P_500),  or simply the S&P, is a stock market index that measures the stock performance of 500 large companies listed on stock exchanges in the United States. It is one of the most commonly followed equity indices.

Schiller P/E is Prof. Shiller's version of the Price to Earnings (P/E) ratio. You can think of P/E as how much would it cost you now to get the earnings of the company for subsequent years. For example, a single share of a company costs \\$40. That company produces \\$2 earnings per share in a year. The P/E ratio will be 40/2 = 20. 

Lets plot some data.

In [4]:
tick_spacing = 20
fig, ax = plt.subplots(1,1, figsize = (10,6))
ax.plot(data.column("Date"), data.column("Shiller P/E"))
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
plt.title("Shiller P/E Quartely")
plt.show();

You can see that the Shiller P/E is at one of its highest levels seen. The only time it excced current levels is the 2004 dot com bubble.

In [5]:
tick_spacing = 20
fig, ax = plt.subplots(1,1, figsize = (10,6))
ax.plot(data.column("Date"), data.column("S&P 500 Closing Price"))
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
plt.title("S&P 500 Quartely Closing Price")
plt.show();

This just shows what we read about so far; the market goes up in the long run. However, we care about actual percentage returns; whether they are high or low or even negative.

For this purpose of predicting returns, we are going to use Shiller P/E as a explantory variable. The idea is to predict the following.

> If I buy the S&P today, given that the P/E is 36.61, what will be my best prediction for the return I get?

## Part 1: Calculating Returns

First lets calcuate returns. The formula for calulating annualized returns for $n$ years is 

$$\left (\frac{\text{Future Price}}{\text{Old Price}} \right )^{\frac{1}{n}} -1$$

We will chose a 10 year period to calcuate subsequent returns (becuase we want to know what we would get in returns if we invest now and wait for 10 years). For example, S&P 500 on 1980-04-01 is \\$102.09. Ten years ahead, on 1990-04-01, the S&P 500 index is at \\$339.94. So the 10 year subsequent returns calcuated on 1980-04-01 are 

$$\left (\frac{339.94}{102.09} \right )^{\frac{1}{10}} -1  = 0.127$$

That represents a 12.7% annualized return.

**Question 1.1:** Calculate 10 year subsequent annualized return for our data. Naturally, we dont have future prices for recent data, so we cant compute that (i.e. cant compute 10 years returns for 2011-07-01 onwards as 10 years have not passed. Store the values in an array called `returns`.


_Hint:_ For each year, we have 4 values; one for each quarter. Think about what slice of the data you need to get the years you want.

<!--
BEGIN QUESTION
name: q1_1
points:
    - 0
    - 0.5
    - 0.75
    - 0.75
manual: false
-->

In [6]:
old = ...
new = ...
returns = ...
returns

In [ ]:
grader.check("q1_1")

**Question 1.2:** Add the returns column to our `data` table with the column name `10 Year Returns`. Multiply by returns you have calculated by 100 to get percentages (since they're easier to interpret that way). The returns array has fewer values than the rows in our data. Use `np.nan` as the return for dates we don't have values for.

_Hint:_ Use [`np.repeat`](https://numpy.org/doc/stable/reference/generated/numpy.repeat.html).

<!--
BEGIN QUESTION
name: q1_2
points: 
    - 0
    - 0
    - 1
manual: false
-->

In [11]:
modified_returns = ...
data = data.with_columns("10 Year Returns", modified_returns)
data

In [ ]:
grader.check("q1_2")

## Part 2: The Setup

Lets plot the data.

In [15]:
fig, ax1 = plt.subplots(1,1, figsize = (10,6))
tick_spacing = 20

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('P/E', color=color)
ax1.plot(data.column("Date"), data.column("Shiller P/E"), color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

ax2 = ax1.twinx() 

color = 'tab:red'
ax2.set_ylabel('10 Year Returns', color=color)  
ax2.plot(data.column("Date"), data.column("10 Year Returns"), color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

fig.tight_layout()  

plt.title("P/E vs 10 Year Returns")
plt.show();

As we expect, we can see an inverse relationship between our valuaion metric P/E and returns. In 2004, at the peak of the dot com bubble, when valuations were high, 10 year subsequent returns were low (actually negative). In 2009, after stocks crashed follwing the recession, stock valuations became very cheap. You can see that those who bought in at low valuations must have made high positive returns. 

Just to show how well valuations track returns (inversly), let's **flip the P/E $y$-axis** (i.e. put smaller values of P/E on top of the axis).

In [16]:
fig, ax1 = plt.subplots(1,1, figsize = (10,6))
tick_spacing = 20

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('P/E', color=color)
ax1.plot(data.column("Date"), data.column("Shiller P/E"), color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
ax1.invert_yaxis()

ax2 = ax1.twinx()  

color = 'tab:red'
ax2.set_ylabel('10 Year Returns', color=color) 
ax2.plot(data.column("Date"), data.column("10 Year Returns"), color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

fig.tight_layout()  

plt.title("P/E vs 10 Year Returns Flipped")
plt.show()

Look at how well it tracks. This shows that valuation is a prime candidate to predict returns. Lets do some regression.

## Part 3: Single Variable Regression

In the plot above, we plotted both P/E and future returns against time. For our regression, we will be using P/E to predict the returns. So P/E is our $x$ variable (independent or explanatory variable). Returns are our $y$ variable (or outcome variable).

Let's understand the relationship between the $x$ and $y$ data columns. In the following plot, each point represents an obervation in time, recording the P/E ratio and the 10 year subsequent returns from that day.

In [56]:
plt.figure(figsize = (10,8))
plt.scatter(data.column("Shiller P/E"), data.column("10 Year Returns"))
plt.xlabel("Shiller P/E")
plt.ylabel("10 Year Returns")
plt.title("Scatter plot of P/E and Returns")
plt.axhline(y=0, color='r', linestyle='-');

This scatterplot provides an even better indication of the strong inverse relationship. In fact, WSJ included a similar image in their article.

<img src="scatterplot.png">

Note that some of the differences in our plot versus the WSJ are becuase they used more data and the scales are different.

**Question 2.1:** Use P/E as an independent variable to predict 10 year returns. Save the regression model to the variable `result`.

Since we have missing returns in the recent 10 years, we will use the `missing = 'drop'` arguement in `sm.OLS()` function.

<!--
BEGIN QUESTION
name: q2_1
points:
    - 0
    - 0
    - 1
    - 1
manual: false
-->

In [57]:
x = ...
y = ...
model = ...
result = ...
result.summary()

In [ ]:
grader.check("q2_1")

<!-- BEGIN QUESTION -->

**Question 2.2:** What is the sign on $\hat{\beta}$? Does it make sense?

<!--
BEGIN QUESTION
name: q2_2
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

**Question 2.3:** Use the estimates for $\hat{\alpha}$ and $\hat{\beta}$ to predict the returns for 2021-01-01. Store it in a variable called `prediction_today`.

_Hint:_ $\hat{y} = \hat{\alpha} + \hat{\beta}x$

<!--
BEGIN QUESTION
name: q2_3
points: 
    - 0
    - 1
manual: false
-->

In [63]:
prediction_today = ...
prediction_today

In [ ]:
grader.check("q2_3")

Before we move on, its important to understand the interpretation of $\hat{\beta}$ and by extension regression here. In this lab, we are using regression as a tool for prediction. We are not making any causal inference.

Causal inference is focused on knowing what happens to $y$ when you change $x$; saying $x$ causes $y$. Prediction is focused on knowing the next $y$ given $x$. Regression is used for both. It is important to know which purpose you are using it for.

Thus, the interpretation of $\hat{\beta}$ here is as follows. **A 1 unit increase in P/E ratio is associated with a -0.4661 change in returns.**

Let's plot the predicted values for all years.

In [67]:
plt.figure(figsize = (10,8))
plt.scatter(data.column("Shiller P/E"), data.column("10 Year Returns"), label = "Actual Returns")
predicted = result.params[0] + result.params[1]*data.column("Shiller P/E")
plt.scatter(data.column("Shiller P/E"),predicted , label = "Predicted Returns")
plt.xlabel("Shiller P/E")
plt.ylabel("10 Year Returns")
plt.title("Actual returns vs Predicted Returns")
plt.legend()
plt.axhline(y=0, color='r', linestyle='-');

## Part 4: Multivariate Regression

As you calculated in Question 2.3, predicted return 10 years from now is roughly 2%. Historical average yearly returns in the long run have been 8%. Based on this, does it seem like a good time to invest?

Think about this. If on 2021-01-01, you decided to sell your investment in S&P 500 based on the 2% prediction, you would have missed out on 5.7% growth in the first quarter of this year (which is 25% annualized). 

So while on the long run P/E valuation is a good metric, in the short run, we need other data.

[Investor allocation to stocks](https://iranthenumbers.wordpress.com/2020/07/04/http-www-philosophicaleconomics-com-2013-12-the-single-greatest-predictor-of-future-stock-market-returns/) (aka _equity allocation_) is another common valuation metric. Intuitively, you can think of it as the percentage of the porfolio people allocation to stocks versus other assets. Mathematically it is as follows: 

$$\text{investor allocation to stocks} = \frac{\text{market value of all stocks}}{\text{market value of all stocks} + \text{total liabilities of all real economic borrowers}}$$

[FRED](https://fred.stlouisfed.org/graph/?g=qis) updates the values quarterly for this metric.

First, lets look at some plots for this.

In [70]:
fig, ax1 = plt.subplots(1,1, figsize = (10,6))
tick_spacing = 20

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Equity Allocation', color=color)
ax1.plot(data.column("Date"), data.column("Equity Allocation"), color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

ax2 = ax1.twinx()

color = 'tab:red'
ax2.set_ylabel('10 Year Returns', color=color) 
ax2.plot(data.column("Date"), data.column("10 Year Returns"), color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

fig.tight_layout() 

plt.title("Equity Allocation vs 10 Year Returns")
plt.show();

In [71]:
fig, ax1 = plt.subplots(1,1, figsize = (10,6))
tick_spacing = 20

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Equity Allocation', color=color)
ax1.plot(data.column("Date"), data.column("Equity Allocation"), color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
ax1.invert_yaxis()

ax2 = ax1.twinx() 

color = 'tab:red'
ax2.set_ylabel('10 Year Returns', color=color)
ax2.plot(data.column("Date"), data.column("10 Year Returns"), color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

fig.tight_layout() 

plt.title("Equity Allocation vs 10 Year Returns Flipped")
plt.show()

In [72]:
plt.figure(figsize = (10,8))
plt.scatter(data.column("Equity Allocation"), data.column("10 Year Returns"))
plt.xlabel("Equity Allocation")
plt.ylabel("10 Year Returns")
plt.title("Scatter plot of Equity Allocation and Returns")
plt.axhline(y=0, color='r', linestyle='-');

So, equity allocation is also a good tracking indicator.

**Question 3.1:** Similar to Question 2.1, use equity allocation as an independent variable to predict 10 year returns. Save the regression model to the variable `result2`.


<!--
BEGIN QUESTION
name: q3_1
points:
    - 0
    - 0
    - 1
    - 1
manual: false
-->

In [74]:
x = ...
y = ...
model = ...
result2 = ...
result2.summary()

In [ ]:
grader.check("q3_1")

**Question 3.2:** Use the estimates for $\hat{\alpha}$ and $\hat{\beta}$ to predict the returns for 2020-10-01. Store it in a variable called `prediction_today2`.

<!--
BEGIN QUESTION
name: q3_2
points:
    - 0
    - 1
manual: false
-->

In [80]:
prediction_today2 = ...
prediction_today2

In [ ]:
grader.check("q3_2")

In [83]:
plt.figure(figsize = (10,8))
plt.scatter(data.column("Equity Allocation"), data.column("10 Year Returns"), label = "Actual Returns")
predicted = result2.params[0] + result2.params[1]*data.column("Equity Allocation")
plt.scatter(data.column("Equity Allocation"),predicted , label = "Predicted Returns")
plt.xlabel("Equity Allocation")
plt.ylabel("10 Year Returns")
plt.title("Actual returns vs Predicted Returns")
plt.legend()
plt.axhline(y=0, color='r', linestyle='-');

**Question 3.3:** Use both P/E and equity allocaition to predict returns. P/E should be to the $\hat{\beta}_1$ coefficient and equity allocation should be $\hat{\beta}_2$. Save the regression model to the variable `result3`.

<!--
BEGIN QUESTION
name: q3_3
points: 
    - 0
    - 0
    - 1
    - 1
    - 1
manual: false
-->

In [85]:
x = ...
y = ...
model = ...
result3 = ...
result3.summary()

In [ ]:
grader.check("q3_3")

**Question 3.4:** Use the estimates for $\hat{\alpha}$, $\hat{\beta_1}$, and $\hat{\beta_2}$to predict the returns for 2020-10-01. Store it in a variable called `prediction_today3`.


<!--
BEGIN QUESTION
name: q3_4
points:
    - 0
    - 1
manual: false
-->

In [92]:
prediction_today3 = ...
prediction_today3

In [ ]:
grader.check("q3_4")

Since we have 2 independent variables, we will need to make a 3D plot to see how our predicted values line up with our actual values.

In [97]:
fig = plt.figure(figsize = (16,12))
ax = fig.add_subplot(projection='3d')

xs = data.column("Shiller P/E")
ys = data.column("Equity Allocation")
zs =  data.column("10 Year Returns")
ax.scatter(xs, ys, zs, marker='o')

predicted = result3.params[0] + result3.params[1]*data.column("Shiller P/E") + \
    result3.params[2]*data.column("Equity Allocation")

xs = data.column("Shiller P/E")
ys = data.column("Equity Allocation")
zs =  predicted
ax.scatter(xs, ys, zs, marker='^')

ax.set_xlabel('Shiller P/E')
ax.set_ylabel('Equity Allocation')
ax.set_zlabel('10 Year Returns')

plt.legend(["Actual", "Predicted"])
plt.show()

## Part 5: Comparing Regressions

Let's compare the regression model with just Equity Allocation versus the model with both P/E and Equity Allocation. The predictions for 2020-10-01 are simlar enough for both. So are these models differnt performance wise?

We will use a metric called Adjusted R-squared to comare. Statsmodels regression results also calulated these for us. You can find Adj. R-squared in the second row from the top on the right. 

We will not formally teach Adjusted R-squared in this class. So an intuitve explaination would suffice. 

The [adjusted R-squared](https://blog.minitab.com/en/adventures-in-statistics-2/multiple-regession-analysis-use-adjusted-r-squared-and-predicted-r-squared-to-include-the-correct-number-of-variables#:~:text=The%20adjusted%20R%2Dsquared%20is,less%20than%20expected%20by%20chance.) is a modified version of R-squared that has been adjusted for the number of predictors (features) in the model. The adjusted R-squared increases only if the new term improves the model more than would be expected by chance. It decreases when a predictor improves the model by less than expected by chance. 

The Adj R-squared for model 2 is 0.827. The Adj. R-squared for model 3 is 0.829. So adding P/E as another variable did not add much to the model in terms of accuracy or prediction power. Instead, it added more complexity and took more compute power. We had very less data, but you can imagine it would be a problem with a lot more data.

A good question to ask is why did the model not improve? Lets look the data more closely.

In [100]:
fig, ax1 = plt.subplots(1,1, figsize = (10,6))
tick_spacing = 20

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Equity Allocation', color=color)
ax1.plot(data.column("Date"), data.column("Equity Allocation"), color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

ax2 = ax1.twinx()  

color = 'tab:red'
ax2.set_ylabel('P/E', color=color)  
ax2.plot(data.column("Date"), data.column("Shiller P/E"), color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

fig.tight_layout()  

plt.title("Equity Allocation vs Shiller P/E")
plt.show();

In [101]:
plt.figure(figsize = (10,8))
plt.scatter(data.column("Shiller P/E"), data.column("Equity Allocation"))
plt.xlabel("Shiller P/E")
plt.ylabel("Equity Allocation")
plt.title("Scatter plot of Shiller P/E and Equity Allocation");

Both Equity Allocation and Shiller P/E track each other pretty closely, as one would expect.

Let's calculate the correlation between the two variables.

In [102]:
# No need to understand the code
corr = np.corrcoef(data.column("Shiller P/E")[:-2], data.column("Equity Allocation")[:-2])[0][1]
corr

In fact, the correlation between P/E and Equity Allocation is quite high; 0.93. This shows that given one of the features, we practically know the other feature. Hence, having both in a model is generally not useful. This problem is called **Multicollinearity**.

Congrats on finishing this lab. Now you can do analysis similar to the WSJ!

---

## Feedback

The last question of this assignment will ask you to complete a short feedback survey, for which you will receive a free point! Please run the cell below to display the form. After you submit, you will receive a codeword from the confirmation page which you can enter below.

In [52]:
from IPython.display import display, IFrame
display(IFrame("https://docs.google.com/forms/d/e/1FAIpQLSe0W7pXSSvmy6T2ap74uXzeNN4nCCy_s8NkLofU"
               "mI2mhQlnvQ/viewform?embedded=true", 800, 600))

Enter your codeword below, assigning at as a string to `feedback_codeword`.

<!--
BEGIN QUESTION
name: feedback
points:
    - 0
    - 1
-->

In [53]:
feedback_codeword = "..."
feedback_codeword

In [ ]:
grader.check("feedback")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)